In [2]:
import pygame
import random
import math

pygame.init()

class DrawInformation:
    BLACK = 0, 0, 0
    WHITE = 255, 255, 255
    GREEN = 0, 255, 0
    RED = 255, 0, 0
    BACKGROUND_COLOR = WHITE

    GRADIENTS = [
        (128, 128, 128),
        (160, 160, 160),
        (192, 192, 192)
    ]

    FONT = pygame.font.SysFont('comicsans', 30)
    LARGE_FONT = pygame.font.SysFont('comicsans', 40)

    SIDE_PAD = 100
    TOP_PAD = 150

    def __init__(self, width, height, lst):
        self.width = width
        self.height = height

        self.window = pygame.display.set_mode((width, height))
        pygame.display.set_caption("Sorting Algorithm Visualization")
        self.set_list(lst)

    def set_list(self, lst):
        self.lst = lst
        self.min_val = min(lst)
        self.max_val = max(lst)

        self.block_width = round((self.width - self.SIDE_PAD) / len(lst))
        self.block_height = math.floor((self.height - self.TOP_PAD) / (self.max_val - self.min_val))
        self.start_x = self.SIDE_PAD // 2

    def draw(self, algo_name, ascending):
        self.window.fill(self.BACKGROUND_COLOR)

        title = self.LARGE_FONT.render(f"{algo_name} - {'Ascending' if ascending else 'Descending'}", 1, self.GREEN)
        self.window.blit(title, (self.width / 2 - title.get_width() / 2, 5))

        controls = self.FONT.render("R - Reset | SPACE - Start Sorting | A - Ascending | D - Descending", 1, self.BLACK)
        self.window.blit(controls, (self.width / 2 - controls.get_width() / 2, 45))

        sorting = self.FONT.render("I - Insertion Sort | B - Bubble Sort | S - Selection Sort | M - Merge Sort | Q - Quick Sort", 1, self.BLACK)
        self.window.blit(sorting, (self.width / 2 - sorting.get_width() / 2, 75))

        self.draw_list()
        pygame.display.update()

    def draw_list(self, color_positions={}, clear_bg=False):
        if clear_bg:
            clear_rect = (self.SIDE_PAD // 2, self.TOP_PAD, self.width - self.SIDE_PAD, self.height - self.TOP_PAD)
            pygame.draw.rect(self.window, self.BACKGROUND_COLOR, clear_rect)

        for i, val in enumerate(self.lst):
            x = self.start_x + i * self.block_width
            y = self.height - (val - self.min_val) * self.block_height

            color = self.GRADIENTS[i % 3]

            if i in color_positions:
                color = color_positions[i]

            pygame.draw.rect(self.window, color, (x, y, self.block_width, self.height))

        if clear_bg:
            pygame.display.update()

    
    def generate_starting_list(n, min_val, max_val):
        lst = []

        for _ in range(n):
            val = random.randint(min_val, max_val)
            lst.append(val)

        return lst

   
    def bubble_sort(self, ascending=True):
        lst = self.lst

        for i in range(len(lst) - 1):
            for j in range(len(lst) - 1 - i):
                num1 = lst[j]
                num2 = lst[j + 1]

                if (num1 > num2 and ascending) or (num1 < num2 and not ascending):
                    lst[j], lst[j + 1] = lst[j + 1], lst[j]
                    self.draw_list({j: self.GREEN, j + 1: self.RED}, True)
                    yield True

        return lst

   
    def insertion_sort(self, ascending=True):
        lst = self.lst

        for i in range(1, len(lst)):
            current = lst[i]

            while True:
                ascending_sort = i > 0 and lst[i - 1] > current and ascending
                descending_sort = i > 0 and lst[i - 1] < current and not ascending

                if not ascending_sort and not descending_sort:
                    break

                lst[i] = lst[i - 1]
                i = i - 1
                lst[i] = current
                self.draw_list({i - 1: self.GREEN, i: self.RED}, True)
                yield True

        return lst

    
    def selection_sort(self, ascending=True):
        lst = self.lst

        for i in range(len(lst)):
            min_idx = i

            for j in range(i + 1, len(lst)):
                if (lst[j] < lst[min_idx] and ascending) or (lst[j] > lst[min_idx] and not ascending):
                    min_idx = j

            lst[i], lst[min_idx] = lst[min_idx], lst[i]
            self.draw_list({i: self.GREEN, min_idx: self.RED}, True)
            yield True

        return lst

    
    def merge_sort(self, ascending=True):
        def merge(arr, left, mid, right):
            n1 = mid - left + 1
            n2 = right - mid

            L = [arr[left + i] for i in range(n1)]
            R = [arr[mid + 1 + i] for i in range(n2)]

            i = j = 0
            k = left

            while i < n1 and j < n2:
                if (L[i] <= R[j] and ascending) or (L[i] >= R[j] and not ascending):
                    arr[k] = L[i]
                    i += 1
                else:
                    arr[k] = R[j]
                    j += 1
                k += 1

            while i < n1:
                arr[k] = L[i]
                i += 1
                k += 1

            while j < n2:
                arr[k] = R[j]
                j += 1
                k += 1

        def merge_sort_helper(arr, left, right):
            if left < right:
                mid = (left + right) // 2

                yield from merge_sort_helper(arr, left, mid)
                yield from merge_sort_helper(arr, mid + 1, right)

                merge(arr, left, mid, right)
                self.draw_list({i: self.GREEN for i in range(left, right + 1)}, True)
                yield True

        lst = self.lst
        yield from merge_sort_helper(lst, 0, len(lst) - 1)
        return lst

    
    def quick_sort(self, ascending=True):
        def partition(arr, low, high):
            pivot = arr[high]
            i = low - 1

            for j in range(low, high):
                if (arr[j] <= pivot and ascending) or (arr[j] >= pivot and not ascending):
                    i += 1
                    arr[i], arr[j] = arr[j], arr[i]
                    self.draw_list({i: self.GREEN, j: self.RED}, True)
                    yield True

            arr[i + 1], arr[high] = arr[high], arr[i + 1]
            self.draw_list({i + 1: self.GREEN, high: self.RED}, True)
            yield True

            return i + 1

        def quick_sort_helper(arr, low, high):
            if low < high:
                pi = yield from partition(arr, low, high)

                yield from quick_sort_helper(arr, low, pi - 1)
                yield from quick_sort_helper(arr, pi + 1, high)

        lst = self.lst
        yield from quick_sort_helper(lst, 0, len(lst) - 1)
        return lst


def main():
    run = True
    clock = pygame.time.Clock()

    n = 50
    min_val = 0
    max_val = 100

    lst = DrawInformation.generate_starting_list(n, min_val, max_val)
    self = DrawInformation(1300, 700, lst)
    sorting = False
    ascending = True

    sorting_algorithm = DrawInformation.bubble_sort
    sorting_algo_name = "Bubble Sort"
    sorting_algorithm_generator = None

    while run:
        clock.tick(60)

        if sorting:
            try:
                next(sorting_algorithm_generator)
            except StopIteration:
                sorting = False
        else:
            self.draw(sorting_algo_name, ascending)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False

            if event.type != pygame.KEYDOWN:
                continue

            if event.key == pygame.K_r:
                lst = DrawInformation.generate_starting_list(n, min_val, max_val)
                self.set_list(lst)
                sorting = False
            elif event.key == pygame.K_SPACE and not sorting:
                sorting = True
                sorting_algorithm_generator = sorting_algorithm(self, ascending)
            elif event.key == pygame.K_a and not sorting:
                ascending = True
            elif event.key == pygame.K_d and not sorting:
                ascending = False
            elif event.key == pygame.K_i and not sorting:
                sorting_algorithm = DrawInformation.insertion_sort
                sorting_algo_name = "Insertion Sort"
            elif event.key == pygame.K_b and not sorting:
                sorting_algorithm = DrawInformation.bubble_sort
                sorting_algo_name = "Bubble Sort"
            elif event.key == pygame.K_s and not sorting:
                sorting_algorithm = DrawInformation.selection_sort
                sorting_algo_name = "Selection Sort"
            elif event.key == pygame.K_m and not sorting:
                sorting_algorithm = DrawInformation.merge_sort
                sorting_algo_name = "Merge Sort"
            elif event.key == pygame.K_q and not sorting:
                sorting_algorithm = DrawInformation.quick_sort
                sorting_algo_name = "Quick Sort"

    pygame.quit()


if __name__ == "__main__":
    main()
